# Model 2

In [24]:
import torch
from tqdm import tqdm
import os
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset, random_split, Dataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
import random
import numpy as np
from collections import defaultdict
from sklearn.utils.class_weight import compute_class_weight
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch.nn import TransformerEncoder, TransformerEncoderLayer

## initiation:

In [9]:
class NumpyFolderDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        """
        Dataset to handle .npy files organized by class in subdirectories.

        Args:
        - root_dir (str): Root directory containing class subdirectories.
        - transform (callable, optional): Transform to apply to each sample.
        """
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []  # List to store file paths and their labels

        # Traverse through the directory structure
        for class_idx, class_name in enumerate(sorted(os.listdir(root_dir))):
            class_dir = os.path.join(root_dir, class_name)
            if os.path.isdir(class_dir):
                for file_name in os.listdir(class_dir):
                    if file_name.endswith('.npy'):
                        file_path = os.path.join(class_dir, file_name)
                        self.samples.append((file_path, class_idx))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        file_path, label = self.samples[idx]
        np_array = np.load(file_path).astype(np.float32)  # Load the .npy file

        if self.transform:
            np_array = self.transform(np_array)
        return np_array, label

# Define transformations
transform = transforms.Compose([
    transforms.Lambda(lambda x: torch.tensor(x)),  # Convert NumPy array to PyTorch tensor
    transforms.Lambda(lambda x: x.unsqueeze(0)),   # Add channel dimension for CNN
])

# Path to your dataset
dataset_path = r'G:\Thesis_Numpy_data_set\2_Class_160'

# Create the dataset
dataset = NumpyFolderDataset(root_dir=dataset_path, transform=transform)

# Split into training and test datasets
train_size = int(0.9 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Verify DataLoader
for batch_data, batch_labels in train_loader:
    print(f"Batch data shape: {batch_data.shape}, Batch labels: {batch_labels.shape}")
    break

Batch data shape: torch.Size([32, 1, 160, 160]), Batch labels: torch.Size([32])


### 1

In [14]:
class ImprovedCNNWithTransformer(nn.Module):
    def __init__(self, num_classes=2, num_transformer_layers=4, num_heads=8):
        super().__init__()

        # Convolutional Layers
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1)

        self.shortcut = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=1, stride=2),
            nn.BatchNorm2d(128)
        )

        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)

        # Transformer Parameters
        self.embed_dim = 128  # Token embedding size
        self.global_pool = nn.AdaptiveAvgPool2d((4, 4))  # Fixed-size output for tokenization

        # Transformer Encoder
        encoder_layer = TransformerEncoderLayer(d_model=self.embed_dim, nhead=num_heads, dim_feedforward=512, dropout=0.4)
        self.transformer = TransformerEncoder(encoder_layer, num_layers=num_transformer_layers)

        # Fully Connected Layers
        self.fc1 = nn.Linear(self.embed_dim, 1024)
        self.fc2 = nn.Linear(1024, num_classes)
        self.dropout = nn.Dropout(0.4)

    def forward(self, x):
        # Block 1
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, 2)

        # Block 2
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, 2)

        # Block 3 with Residual
        shortcut = self.shortcut(x)  # Downsample shortcut
        x = F.relu(self.bn3(self.conv3(x)) + shortcut)

        # Global Pooling
        x = self.global_pool(x)  # Shape: [batch_size, 128, 4, 4]
        batch_size, channels, height, width = x.size()

        # Prepare Transformer Input
        x = x.view(batch_size, channels, -1).permute(0, 2, 1)  # Shape: [batch, 16, 128]

        # Transformer Encoder
        x = self.transformer(x)  # Shape: [batch, 16, 128]
        x = x.mean(dim=1)  # Aggregate token representations (Shape: [batch, 128])

        # Fully Connected Layers
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x
model = ImprovedCNNWithTransformer()

## Eval Section:

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

criterion = nn.CrossEntropyLoss()

# Updated optimizer with weight decay
# optimizer = optim.Adam(model.parameters(), lr=3e-4)
optimizer = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-2)
num_epochs = 50  # Adjust as needed
model.to(device)  # Ensure the model is moved to the device

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    # Training loop
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to device

        # Forward pass and optimization
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Track loss and accuracy
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_samples += labels.size(0)

    # Print detailed epoch metrics
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = correct_predictions / total_samples
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, "
          f"Accuracy: {epoch_accuracy:.2%}, "
          f"LR: {optimizer.param_groups[0]['lr']:.6f}")

# Evaluation loop
model.eval()
correct_predictions = 0
total_samples = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to device
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_samples += labels.size(0)

# Print test accuracy
test_accuracy = correct_predictions / total_samples
print(f"Test Accuracy: {test_accuracy:.2%}")


Epoch [1/50], Loss: 0.4567, Accuracy: 79.13%, LR: 0.000300
Epoch [2/50], Loss: 0.3986, Accuracy: 82.41%, LR: 0.000300
Epoch [3/50], Loss: 0.3769, Accuracy: 83.69%, LR: 0.000300
Epoch [4/50], Loss: 0.3217, Accuracy: 86.61%, LR: 0.000300
Epoch [5/50], Loss: 0.3055, Accuracy: 87.04%, LR: 0.000300
Epoch [6/50], Loss: 0.3002, Accuracy: 88.03%, LR: 0.000300
Epoch [7/50], Loss: 0.2689, Accuracy: 88.75%, LR: 0.000300
Epoch [8/50], Loss: 0.2839, Accuracy: 88.25%, LR: 0.000300
Epoch [9/50], Loss: 0.2684, Accuracy: 88.53%, LR: 0.000300
Epoch [10/50], Loss: 0.2267, Accuracy: 90.03%, LR: 0.000300
Epoch [11/50], Loss: 0.2048, Accuracy: 91.67%, LR: 0.000300
Epoch [12/50], Loss: 0.1860, Accuracy: 92.45%, LR: 0.000300
Epoch [13/50], Loss: 0.1926, Accuracy: 92.24%, LR: 0.000300
Epoch [14/50], Loss: 0.2020, Accuracy: 91.45%, LR: 0.000300
Epoch [15/50], Loss: 0.1734, Accuracy: 93.16%, LR: 0.000300
Epoch [16/50], Loss: 0.1643, Accuracy: 94.16%, LR: 0.000300
Epoch [17/50], Loss: 0.1572, Accuracy: 93.87%, LR

In [25]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

def evaluate_model(model, test_loader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc="Evaluating"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)

            # Get predictions
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Convert to numpy arrays for evaluation
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)

    # Overall accuracy
    accuracy = accuracy_score(all_labels, all_preds)
    print(f"Overall Test Accuracy: {accuracy:.4f}")

    # Per-class accuracy
    class_accuracy = np.diag(confusion_matrix(all_labels, all_preds)) / np.bincount(all_labels)
    for i, acc in enumerate(class_accuracy):
        print(f"Class {i} Accuracy: {acc:.4f}")

    # Classification report
    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds, target_names=dataset.classes))

    # Confusion matrix
    print("\nConfusion Matrix:")
    print(confusion_matrix(all_labels, all_preds))

    return accuracy

# Test the model
test_accuracy = evaluate_model(model, test_loader, device)

Evaluating: 100%|██████████| 5/5 [00:00<00:00, 11.12it/s]

Overall Test Accuracy: 0.9295
Class 0 Accuracy: 0.8974
Class 1 Accuracy: 0.9615

Classification Report:


AttributeError: 'NumpyFolderDataset' object has no attribute 'classes'